In [1]:
# Modules imported
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

In [7]:
# Read CSV file
df = pd.read_csv('Australian Vehicle Prices.csv')

In [8]:
# Data cleaning

# Drop rows where all values are NaN
df = df.dropna(how='all')

# Convert Year to integer
df['Year'] = df['Year'].astype(int)

# Replace '-' and '- / -' with NaN
df = df.replace('-', np.nan)
df = df.replace('- / -', np.nan)

# Drop the Car/Suv column
df = df.drop(columns=['Car/Suv'])

# Drop the Title column
df = df.drop(columns=['Title'])

# Special case
df.loc[df['CylindersinEngine'].str.contains('2 L', na=False), 'CylindersinEngine'] = np.nan # Wrong value, replaced by NaN
# Extract numeric part from CylindersinEngine
df['CylindersinEngine'] = df['CylindersinEngine'].str.extract(r'(\d+)\s*cyl')  # Extract numbers
df['CylindersinEngine'] = df['CylindersinEngine'].astype(float)  # Convert to float for NaN handling

# Extract numeric capacity from Engine
df['EngineCapacity'] = df['Engine'].str.extract(r'([\d.]+)\s*L')  # Extract engine capacity
df['EngineCapacity'] = df['EngineCapacity'].astype(float)  # Convert to float
# Replace zeroes with NaN
df['EngineCapacity'] = df['EngineCapacity'].replace(0, np.nan)

# Drop Engine
df = df.drop(columns=['Engine'])

# Remove units and convert to float
df['FuelConsumptionPer100km'] = df['FuelConsumption'].str.replace(r'L / 100 km', '', regex=True).astype(float)
# Replace 0 with NaN
df['FuelConsumptionPer100km'] = df['FuelConsumptionPer100km'].replace(0, np.nan)

# Drop FuelConsumption
df = df.drop(columns=['FuelConsumption'])

# Convert to numeric
df['Kilometres'] = pd.to_numeric(df['Kilometres'], errors='coerce')

# Extract only the exterior color
# Removing interior color because over 7000 missing values, and contains wrong data
df['ColourExt'] = df['ColourExtInt'].str.split(' / ', expand=True)[0]
# Replace '-' with NaN
df['ColourExt'] = df['ColourExt'].replace('-', np.nan)
# List of invalid colors
invalid_colors = ['5 years', '3 years', '6 years', '7 years', '2 years', '4 years', '1 years']
# Replace invalid values with 'Other'
df['ColourExt'] = df['ColourExt'].replace(invalid_colors, 'Other')

# Drop the ColourExtInt column
df = df.drop(columns=['ColourExtInt'])

# Convert to 'object' type
df['CylindersinEngine'] = df['CylindersinEngine'].apply(lambda x: str(x) if pd.notnull(x) else x)


# Extract seat values from the Doors column
df['ExtractedSeats'] = df['Doors'].str.extract(r'(\d+)\s*Seats').astype(float)
# Fill missing values in Seats
df['Seats'] = df['Seats'].fillna(df['ExtractedSeats'])
# Set the corresponding rows in Doors to NaN
df.loc[df['ExtractedSeats'].notna(), 'Doors'] = np.nan
# Drop the temporary ExtractedSeats column
df = df.drop(columns=['ExtractedSeats'])

# Remove the word 'Seats'
df['Seats'] = df['Seats'].astype(str).str.replace(r'\s*Seats', '', regex=True).str.strip()
# Convert to numeric
df['Seats'] = pd.to_numeric(df['Seats'], errors='coerce')

# Remove the word 'Doors'
df['Doors'] = df['Doors'].astype(str).str.replace(r'\s*Doors', '', regex=True).str.strip()
# Convert to numeric
df['Doors'] = pd.to_numeric(df['Doors'], errors='coerce')

# Convert to 'object' type
df['Doors'] = df['Doors'].apply(lambda x: str(x) if pd.notnull(x) else x)

# Convert to 'object' type
df['Seats'] = df['Seats'].apply(lambda x: str(x) if pd.notnull(x) else x)

# Replace "POA" with NaN
df['Price'] = df['Price'].replace('POA', np.nan)
# Convert Price to numeric
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

# Split Location into City and State
df[['City', 'State']] = df['Location'].str.split(', ', expand=True)
# Drop the Location column
df = df.drop(columns=['Location'])
# Replacing 'AU-VIC' with 'VIC'
df['State'] = df['State'].replace('AU-VIC', 'VIC')

# Define bins and labels
bins = [1939, 2000, 2005, 2011, 2016, 2019, 2023]
labels = [
    "1940-2000",
    "2001-2005",
    "2006-2011",
    "2012-2016",
    "2017-2019",
    "2020-2023"
]
# Create the bins
df['YearRanges'] = pd.cut(df['Year'], bins=bins, labels=labels, right=True)
# Convert to 'object' type
df['YearRanges'] = df['YearRanges'].astype(str)

# Drop the Year column
df = df.drop(columns=['Year'])

<ipython-input-8-20d148d96e17>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Year'] = df['Year'].astype(int)


In [9]:
# Checking the year ranges
print('Possible values of', df['YearRanges'].value_counts(), '\n')

Possible values of YearRanges
2020-2023    4880
2012-2016    4630
2017-2019    4429
2006-2011    2166
2001-2005     465
1940-2000     163
Name: count, dtype: int64 



In [3]:
# We decided to have 4 YearRanges instead of 6 to make it balanced for classification
df['YearRanges'] = df['YearRanges'].replace(['1940-2000', '2001-2005', '2006-2011'], "Before 2012")

In [10]:
# Information about the dataset

print('Number of rows and columns:', df.shape, '\n') # Number of rows and columns
print('Column names:', df.columns, '\n') # Column names

# Checking for missing values
print('Missing values:')
print(df.isnull().sum(), '\n')

# Data types of columns
print('Data types:')
print(df.dtypes, '\n')

# df.info()
# df.describe() # For numerical columns

Number of rows and columns: (16733, 18) 

Column names: Index(['Brand', 'Model', 'UsedOrNew', 'Transmission', 'DriveType', 'FuelType',
       'Kilometres', 'CylindersinEngine', 'BodyType', 'Doors', 'Seats',
       'Price', 'EngineCapacity', 'FuelConsumptionPer100km', 'ColourExt',
       'City', 'State', 'YearRanges'],
      dtype='object') 

Missing values:
Brand                         0
Model                         0
UsedOrNew                     0
Transmission                251
DriveType                     0
FuelType                    637
Kilometres                  591
CylindersinEngine          1781
BodyType                    281
Doors                      1674
Seats                      1633
Price                        52
EngineCapacity             1795
FuelConsumptionPer100km    2074
ColourExt                   395
City                        449
State                       449
YearRanges                    0
dtype: int64 

Data types:
Brand                       object
Mo

In [4]:
# Export DataFrame to CSV
df.to_csv("cleaned_australian_vehicle_prices.csv", index=False)

In [12]:
# # Cell for testing

# # # Checking the possible values of Brand
# print('Possible values of', df['Brand'].value_counts(), '\n')

# # Checking the possible values of Model
# print('Possible values of', df['Model'].value_counts(), '\n')

# # # Checking the possible values of UsedOrNew
# print('Possible values of', df['UsedOrNew'].value_counts(), '\n')

# # # Checking the possible values of Transmission
# print('Possible values of', df['Transmission'].value_counts(), '\n')

# # # Checking the possible values of DriveType
# print('Possible values of', df['DriveType'].value_counts(), '\n')

# # # Checking the possible values of FuelType
# print('Possible values of', df['FuelType'].value_counts(), '\n')

# # # Checking the possible values of FuelConsumptionPer100km
# print('Possible values of', df['FuelConsumptionPer100km'].value_counts(), '\n')

# # # Checking the possible values of Kilometres
# print('Possible values of', df['Kilometres'].value_counts(), '\n')

# # # Checking the possible values of City
# print('Possible values of', df['City'].value_counts(), '\n')

# # # Checking the possible values of State
# print('Possible values of', df['State'].value_counts(), '\n')

# # Checking the possible values of CylindersinEngine
# print('Possible values of', df['CylindersinEngine'].value_counts(), '\n')

# # # Checking the possible values of BodyType
# print('Possible values of', df['BodyType'].value_counts(), '\n')

# # # Checking the possible values of Doors
# print('Possible values of', df['Doors'].value_counts(), '\n')

# # # Checking the possible values of Seats
# print('Possible values of', df['Seats'].value_counts(), '\n')

# # # Checking the possible values of Price
# print('Possible values of', df['Price'].value_counts(), '\n')

# # # Checking the possible values of EngineCapacity
# print('Possible values of', df['EngineCapacity'].value_counts(), '\n')

# # # Checking the possible values of ColourExt
# print('Possible values of', df['ColourExt'].value_counts(), '\n')